<a href="https://colab.research.google.com/github/MarcFriz/Kaggle-Elo_Merchant_Category_Recommendation/blob/master/Elo_merchant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# dependency installs
!pip install pandas
!pip install numpy
!pip install keras
!pip install kaggle


    100% |████████████████████████████████| 61kB 3.9MB/s 
    100% |████████████████████████████████| 245kB 11.6MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [0]:
# import dependencies
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [26]:
# kaggle credentials
from google.colab import files
files.upload()

{}

In [0]:
# test credentials
ls -1ha kaggle.json

kaggle.json


In [0]:
# File configuration
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [0]:
# download dataset
!kaggle competitions download -c elo-merchant-category-recommendation

 97% 524M/538M [00:06<00:00, 51.9MB/s]
100% 538M/538M [00:07<00:00, 80.3MB/s]
 74% 9.00M/12.2M [00:00<00:00, 31.2MB/s]
100% 12.2M/12.2M [00:00<00:00, 38.1MB/s]
 90% 44.0M/48.9M [00:01<00:00, 22.6MB/s]
100% 48.9M/48.9M [00:01<00:00, 44.2MB/s]
  0% 0.00/838k [00:00<?, ?B/s]
100% 838k/838k [00:00<00:00, 185MB/s]
  0% 0.00/1.09M [00:00<?, ?B/s]
100% 1.09M/1.09M [00:00<00:00, 183MB/s]
  0% 0.00/2.93M [00:00<?, ?B/s]
100% 2.93M/2.93M [00:00<00:00, 134MB/s]
  0% 0.00/17.2k [00:00<?, ?B/s]
100% 17.2k/17.2k [00:00<00:00, 13.5MB/s]


In [0]:
# unzip
!ls
!unzip historical_transactions.csv.zip
!unzip merchants.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip
!unzip new_merchant_transactions.csv.zip
!unzip sample_submission.csv.zip

Data_Dictionary.xlsx		   sample_data
historical_transactions.csv.zip    sample_submission.csv.zip
kaggle.json			   test.csv.zip
merchants.csv.zip		   train.csv.zip
new_merchant_transactions.csv.zip
Archive:  historical_transactions.csv.zip
  inflating: historical_transactions.csv  
Archive:  merchants.csv.zip
  inflating: merchants.csv           
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  new_merchant_transactions.csv.zip
  inflating: new_merchant_transactions.csv  
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [0]:
# first look
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_merchants = pd.read_csv('merchants.csv')
df_historical_transactions = pd.read_csv('historical_transactions.csv')
df_new_merchant_transactions = pd.read_csv('new_merchant_transactions.csv')
df_sample_submission = pd.read_csv('sample_submission.csv')

In [0]:
print(df_train.shape)
df_train.head()

(201917, 6)


,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [0]:
print(df_merchants.shape)
df_merchants.head()

(334696, 22)


,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [0]:
print(df_historical_transactions.shape)
df_historical_transactions.head()

(29112361, 14)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [0]:
print(df_new_merchant_transactions.shape)
df_new_merchant_transactions.head()

(1963031, 14)


,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [0]:
print(df_sample_submission.shape)
df_sample_submission.head()

(123623, 2)


,card_id,target
0,C_ID_0ab67a22ab,0
1,C_ID_130fd0cbdd,0
2,C_ID_b709037bc5,0
3,C_ID_d27d835a9f,0
4,C_ID_2b5e3df5c2,0
